# Cleaning the data

In [1]:
import numpy as np
import pandas as pd

## Downloading the data

Thanks to @chanronnie's [scraping work](https://github.com/chanronnie/Olympics).

In [2]:
!curl https://raw.githubusercontent.com/chanronnie/Olympics/main/data/athletes.csv -o data/athletes.csv
!curl https://raw.githubusercontent.com/chanronnie/Olympics/main/data/athletes_roles.csv -o data/athletes_roles.csv
!curl https://raw.githubusercontent.com/chanronnie/Olympics/main/data/noc_countries.csv -o data/noc_countries.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 72.9M  100 72.9M    0     0  46.2M      0  0:00:01  0:00:01 --:--:-- 46.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7388k  100 7388k    0     0  17.8M      0 --:--:-- --:--:-- --:--:-- 17.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3523  100  3523    0     0  12398      0 --:--:-- --:--:-- --:--:-- 12448


## Reading in python

In [3]:
athletes = pd.read_csv('data/athletes.csv')
athletes_roles = pd.read_csv('data/athletes_roles.csv')
noc_countries = pd.read_csv('data/noc_countries.csv')
host_cities = pd.read_csv('data/host_cities_updated.csv')

In [4]:
athletes.head() # data on athletes for each event

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,medal
0,131892,Meryem Erdoğan,Female,24 April 1990,NaN,172 cm,55 kg,Türkiye,2016 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",NaN
1,131892,Meryem Erdoğan,Female,24 April 1990,NaN,172 cm,55 kg,Türkiye,2020 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",NaN
2,131892,Meryem Erdoğan,Female,24 April 1990,NaN,172 cm,55 kg,Türkiye,2020 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",NaN
3,4300,Maurice Maina,Male,1 January 1963,NaN,158 cm,47 kg,Kenya,1988 Summer Olympics,KEN,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",NaN
4,4300,Maurice Maina,Male,1 January 1963,NaN,158 cm,47 kg,Kenya,1988 Summer Olympics,KEN,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",NaN


In [5]:
athletes_roles.head() # data on athletes roles

,id,name,roles
0,131892,Meryem Erdoğan,Competed in Olympic Games
1,4300,Maurice Maina,Competed in Olympic Games
2,60239,Stanislav Tůma,Competed in Olympic Games
3,129369,Eunice Kirwa,Competed in Olympic Games
4,142670,Sinem Kurtbay,Competed in Olympic Games


In [6]:
noc_countries.head() # data on IOC "NOC" identifier to link to a country

,noc,country
0,AFG,Afghanistan
1,ALB,Albania
2,ALG,Algeria
3,ASA,American Samoa
4,AND,Andorra


In [7]:
host_cities.head() # data on hosted events

,game,host_city,noc
0,2034 Winter Olympics,Salt Lake City,USA
1,2032 Summer Olympics,Brisbane,AUS
2,2030 Winter Olympics,French Alps,FRA
3,2028 Summer Olympics,Los Angeles,USA
4,2026 Winter Olympics,Milano-Cortina d'Ampezzo,ITA


## Cleaning the data utilities

In [8]:
SI_UNITS = ['si', 'cm', 'kg'] # I convert string to numbers in standard unit I defined here

def to_si(element, unit_check=True):
    if type(element) is float:
        return element
    if type(element) is int:
        return element
    if type(element) is str:
        if unit_check:
            el_split = element.split(' ')
            unit = el_split[-1]
            value = el_split[:-1]
        else:
            unit = 'si' # Assume correct unit
            value = element
        
        if len(value) == 1:
            value = value[0]
        value = to_number(value)
        
        if unit in SI_UNITS:
            return value
        else:
            raise Exception(f'Unknown unit "{unit}"')

In [9]:
def to_number(element):
    if type(element) is float:
        return element
    if type(element) is int:
        return element
    if type(element) is str:
        if element=='':
            return np.nan
        
        if '-' in element:
            return to_number(element.split('-'))
        if ',' in element:
            return to_number(element.split(','))
        
        try:
            if '.' in element:
                return float(element)
            return int(element)
        except ValueError:
            print(f'Warning unwanted characters in number {element}.')
            return to_number(''.join(filter(str.isnumeric, element)))
    if type(element) is list:
        return np.average([to_number(e) for e in element])

In [10]:
def to_date(element):
    if type(element) is pd.Timestamp:
        return element
    if type(element) is float:
        if not np.isnan(element):
            Exception('not nan {element}')
        return np.nan
    if type(element) is str:
        if not any(char.isdigit() for char in element):
            return np.nan
        if '(' in element:
            element = element.replace('(','').replace(')','')
        
        if 'c.' in element:
            element = element.replace('c.', 'circa')
        if 'circa' in element:
            key = 'circa '
            i = element.find(key) + len(key)
            return to_date(element[i:])
        
        if '-' in element:
            element = element.replace('-', ' or ')
        if ' or ' in element:
            key = ' or '
            i = element.find(key) - len(key)
            element = [element[i:i+4], element[i+2*len(key):i+4+2*len(key)]]
            element = to_number(element)
            return to_date(element)
        
        element_split = element.split(' ')
        if len(element_split) == 3:
            d, m, y = element_split
            try:
                return pd.to_datetime(f'{d} {m} {y}', format='%d %B %Y')
            except ValueError:
                print(f'Warning unwanted characters in date {element}.')
                pd.to_datetime(f'{to_number(y)}', format='%Y')
        elif len(element_split) == 1:
            try:
                return pd.to_datetime(f'{element_split[0]}', format='%Y')
            except ValueError:
                print(f'Warning unwanted characters in date {element}.')
                pd.to_datetime(f'{to_number(element)}', format='%Y')
        elif len(element_split) == 2:
            m, y = element_split
            try:
                return pd.to_datetime(f'{m} {y}', format='%B %Y')
            except ValueError:
                print(f'Warning unwanted characters in date {element}.')
                pd.to_datetime(f'{to_number(element)}', format='%Y')
        return np.nan
    # raise Exception(f'Bad type {type(element)} from {element} ')

In [11]:
SEASONS = ('Summer', 'Winter')

def sep_game(element):
    element.replace('  ', ' ')
    body = element.split(' ')[1:]
    year = to_number(element[:4])
    season = ' '.join(body[:-1])
    comp_body = body[-1:]
    if season not in SEASONS:
        if season:
            comp_body.insert(0, season)
        season = 'Other'
    competition = ' '.join(comp_body)
    return year, season, competition

## Games & host

In [12]:
host_cities['year'], host_cities['season'], host_cities['competition'] = zip(*host_cities['game'].apply(sep_game))

In [13]:
host_cities.to_csv('data/games.csv', index=False)

## Athletes

In [14]:
athletes['born'] = athletes['born'].apply(to_date)
athletes['died'] = athletes['died'].apply(to_date)
athletes['height'] = athletes['height'].apply(to_si)
athletes['weight'] = athletes['weight'].apply(to_si)

Warning unwanted characters in date In the 1970s.
Warning unwanted characters in number 1970s.
Warning unwanted characters in date In the 1970s.
Warning unwanted characters in number 1970s.
Warning unwanted characters in date In the 1990s.
Warning unwanted characters in number 1990s.
Warning unwanted characters in date In the 1990s.
Warning unwanted characters in number 1990s.
Warning unwanted characters in date Prior to 2013.
Warning unwanted characters in date Prior to 2013.
Warning unwanted characters in date Prior to 2013.
Warning unwanted characters in date before 2010.
Warning unwanted characters in number before 2010.
Warning unwanted characters in date before 2010.
Warning unwanted characters in number before 2010.
Warning unwanted characters in date Prior to 2007.
Warning unwanted characters in date Prior to 2007.
Warning unwanted characters in date Prior to 2007.
Warning unwanted characters in date Prior to 2014.
Warning unwanted characters in date Prior to 2014.
Warning unwa

Warning unwanted characters in date before 2006.
Warning unwanted characters in number before 2006.
Warning unwanted characters in date before 2006.
Warning unwanted characters in number before 2006.
Warning unwanted characters in date Prior to 2008.
Warning unwanted characters in date Prior to 2008.
Warning unwanted characters in date Prior to 2008.
Warning unwanted characters in date before 2010.
Warning unwanted characters in number before 2010.
Warning unwanted characters in date before 2010.
Warning unwanted characters in number before 2010.
Warning unwanted characters in date before 2013.
Warning unwanted characters in number before 2013.
Warning unwanted characters in date before 2013.
Warning unwanted characters in number before 2013.
Warning unwanted characters in date Prior to 2012.
Warning unwanted characters in date Prior to 2012.
Warning unwanted characters in date Prior to 2014.
Warning unwanted characters in date Prior to 2014.
Warning unwanted characters in date Prior t

Warning unwanted characters in date Prior to 2002.
Warning unwanted characters in date Prior to 2002.
Warning unwanted characters in date Prior to 2013.
Warning unwanted characters in date Prior to 2013.
Warning unwanted characters in date Prior to 2009.
Warning unwanted characters in date Prior to 2009.
Warning unwanted characters in date Before 2011.
Warning unwanted characters in number Before 2011.
Warning unwanted characters in date Before 2011.
Warning unwanted characters in number Before 2011.
Warning unwanted characters in date before 2021.
Warning unwanted characters in number before 2021.
Warning unwanted characters in date before 2021.
Warning unwanted characters in number before 2021.
Warning unwanted characters in date before 2021.
Warning unwanted characters in number before 2021.
Warning unwanted characters in date before 2021.
Warning unwanted characters in number before 2021.
Warning unwanted characters in date Prior to 2012.
Warning unwanted characters in date Prior t

In [15]:
athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476348 entries, 0 to 476347
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   id      476348 non-null  int64         
 1   name    476348 non-null  object        
 2   gender  476348 non-null  object        
 3   born    466954 non-null  datetime64[ns]
 4   died    115968 non-null  datetime64[ns]
 5   height  348837 non-null  float64       
 6   weight  339742 non-null  float64       
 7   team    476348 non-null  object        
 8   game    476348 non-null  object        
 9   noc     476348 non-null  object        
 10  sport   476348 non-null  object        
 11  event   476348 non-null  object        
 12  medal   66026 non-null   object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(8)
memory usage: 47.2+ MB


In [16]:
athletes.to_csv('data/athletes_updated.csv', index=False)

In [17]:
athletes = pd.read_csv('data/athletes_updated.csv')
athletes_roles = pd.read_csv('data/athletes_roles.csv')
noc_countries = pd.read_csv('data/noc_countries.csv')
host_cities = pd.read_csv('data/games.csv')

In [18]:
athletes_roles = athletes_roles.drop('name', axis=1)
athletes = pd.merge(athletes, athletes_roles, on='id', how='left')
athletes = pd.merge(athletes, noc_countries, on='noc', how='left')
host_cities = pd.merge(host_cities, noc_countries, on='noc', how='left')
host_cities = host_cities.rename(columns={"noc": "host_noc", "country": "host_country"})
athletes = pd.merge(athletes, host_cities, on='game', how='left')

In [19]:
athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476348 entries, 0 to 476347
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            476348 non-null  int64  
 1   name          476348 non-null  object 
 2   gender        476348 non-null  object 
 3   born          466954 non-null  object 
 4   died          115968 non-null  object 
 5   height        348837 non-null  float64
 6   weight        339742 non-null  float64
 7   team          476348 non-null  object 
 8   game          476348 non-null  object 
 9   noc           476348 non-null  object 
 10  sport         476348 non-null  object 
 11  event         476348 non-null  object 
 12  medal         66026 non-null   object 
 13  roles         476348 non-null  object 
 14  country       476348 non-null  object 
 15  host_city     476348 non-null  object 
 16  host_noc      476348 non-null  object 
 17  year          476348 non-null  int64  
 18  seas

In [20]:
athletes

,id,name,gender,born,died,height,weight,team,game,noc,...,event,medal,roles,country,host_city,host_noc,year,season,competition,host_country
0,131892,Meryem Erdoğan,Female,1990-04-24,NaN,172.0,55.0,Türkiye,2016 Summer Olympics,TUR,...,"Athletics, Marathon, Women(Olympic)",NaN,Competed in Olympic Games,Türkiye,Rio de Janeiro,BRA,2016,Summer,Olympics,Brazil
1,131892,Meryem Erdoğan,Female,1990-04-24,NaN,172.0,55.0,Türkiye,2020 Summer Olympics,TUR,...,"Athletics, Marathon, Women(Olympic)",NaN,Competed in Olympic Games,Türkiye,Tokyo,JPN,2020,Summer,Olympics,Japan
2,131892,Meryem Erdoğan,Female,1990-04-24,NaN,172.0,55.0,Türkiye,2020 Summer Olympics,TUR,...,"Athletics, Marathon, Women(Olympic)",NaN,Competed in Olympic Games,Türkiye,Tokyo,JPN,2020,Summer,Olympics,Japan
3,4300,Maurice Maina,Male,1963-01-01,NaN,158.0,47.0,Kenya,1988 Summer Olympics,KEN,...,"Boxing, Light-Flyweight, Men(Olympic)",NaN,Competed in Olympic Games,Kenya,Seoul,KOR,1988,Summer,Olympics,Republic of Korea
4,4300,Maurice Maina,Male,1963-01-01,NaN,158.0,47.0,Kenya,1988 Summer Olympics,KEN,...,"Boxing, Light-Flyweight, Men(Olympic)",NaN,Competed in Olympic Games,Kenya,Seoul,KOR,1988,Summer,Olympics,Republic of Korea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476343,20989,Caitlin Bilodeaux-Banos,Female,1965-03-17,NaN,170.0,64.0,United States,1988 Summer Olympics,USA,...,"Fencing, Foil, Individual, Women(Olympic)",NaN,Competed in Olympic Games,United States,Seoul,KOR,1988,Summer,Olympics,Republic of Korea
476344,20989,Caitlin Bilodeaux-Banos,Female,1965-03-17,NaN,170.0,64.0,United States,1988 Summer Olympics,USA,...,"Fencing, Foil, Team, Women(Olympic)",NaN,Competed in Olympic Games,United States,Seoul,KOR,1988,Summer,Olympics,Republic of Korea
476345,20989,Caitlin Bilodeaux-Banos,Female,1965-03-17,NaN,170.0,64.0,United States,1992 Summer Olympics,USA,...,"Fencing, Foil, Individual, Women(Olympic)",NaN,Competed in Olympic Games,United States,Barcelona,ESP,1992,Summer,Olympics,Spain
476346,20989,Caitlin Bilodeaux-Banos,Female,1965-03-17,NaN,170.0,64.0,United States,1992 Summer Olympics,USA,...,"Fencing, Foil, Team, Women(Olympic)",NaN,Competed in Olympic Games,United States,Barcelona,ESP,1992,Summer,Olympics,Spain


In [21]:
athletes.to_csv('data/athletes_full.csv', index=False)

In [22]:
!head data/athletes_full.csv

id,name,gender,born,died,height,weight,team,game,noc,sport,event,medal,roles,country,host_city,host_noc,year,season,competition,host_country
131892,Meryem Erdoğan,Female,1990-04-24,,172.0,55.0,Türkiye,2016 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",,Competed in Olympic Games,Türkiye,Rio de Janeiro,BRA,2016,Summer,Olympics,Brazil
131892,Meryem Erdoğan,Female,1990-04-24,,172.0,55.0,Türkiye,2020 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",,Competed in Olympic Games,Türkiye,Tokyo,JPN,2020,Summer,Olympics,Japan
131892,Meryem Erdoğan,Female,1990-04-24,,172.0,55.0,Türkiye,2020 Summer Olympics,TUR,Athletics,"Athletics, Marathon, Women(Olympic)",,Competed in Olympic Games,Türkiye,Tokyo,JPN,2020,Summer,Olympics,Japan
4300,Maurice Maina,Male,1963-01-01,,158.0,47.0,Kenya,1988 Summer Olympics,KEN,Boxing,"Boxing, Light-Flyweight, Men(Olympic)",,Competed in Olympic Games,Kenya,Seoul,KOR,1988,Summer,Olympics,Republic of Korea
4300,Maurice Maina,Male,196